# Melody Generator

This notebook generates a melody once you load a model and the dataset (pitchbased_notes). The latter is used to reproduce X_seed which functions as a novel prompt to the model.

In [1]:
!pip install music21

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pickle
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split
from keras.models import load_model
from tensorflow.keras.utils import to_categorical
import music21
from music21 import *

In [3]:
def sequence_splitterv1(integer_encoded_sequence, window_size):
    network_input = []
    network_output = []
    n_vocab = len(set(integer_encoded_sequence))

    for i in range(0, len(integer_encoded_sequence) - window_size, 1):
        sequence_in = integer_encoded_sequence[i:i + window_size]
        sequence_out = integer_encoded_sequence[i + window_size]
        network_input.append(sequence_in)
        network_output.append(sequence_out)

    n_patterns = len(network_input)
    
    network_input = np.reshape(network_input, (n_patterns, window_size, 1))
    
    # normalize input
    network_input = network_input / float(n_vocab)
    network_output = to_categorical(network_output)
    
    return network_input, network_output

In [4]:
def sequence_splitterv2(notes, window_size):
    features = []
    targets = []
    
    for i in range(0, len(notes) - window_size, 1):
        feature = notes[i:i + window_size]
        target = notes[i + window_size]
        features.append([mapping[j] for j in feature])
        targets.append(mapping[target])
        
    
    return features, targets

In [23]:
with open('/content/pitchbased_notes', 'rb') as f:
    pitchbased_notes = pickle.load(f)

In [24]:
token_size = len(pitchbased_notes)
type_size = len(set(pitchbased_notes))
print(f'Total number of notes: {token_size}')
print(f'Total number of distinct notes: {type_size}')

Total number of notes: 48686
Total number of distinct notes: 331


In [25]:
rare_note = []
for index, (key, value) in enumerate(Counter(pitchbased_notes).items()):
    if value < 20:
        m =  key
        rare_note.append(m)
        
print(f'Total number of notes that occur less than 40 times: {len(rare_note)}')
    
for element in pitchbased_notes:
    if element in rare_note:
        pitchbased_notes.remove(element)

print(f'Number of notes after cleaning: {len(pitchbased_notes)}')

Total number of notes that occur less than 40 times: 161
Number of notes after cleaning: 48086


In [26]:
unique_notes = sorted(list(set(pitchbased_notes)))

#Building dictionary to access the vocabulary from indices and vice versa
mapping = dict((c, i) for i, c in enumerate(unique_notes))
reverse_mapping = dict((i, c) for i, c in enumerate(unique_notes))

In [27]:
window_size = 80

features, targets = sequence_splitterv2(pitchbased_notes, window_size)

In [28]:
targets_length = len(targets)
print("Total number of sequences in the dataset:", targets_length)

Total number of sequences in the dataset: 48006


In [29]:
# reshape X and normalize
X = (np.reshape(features, (targets_length, window_size, 1)))/ float(type_size)
# one hot encode the output variable
y = to_categorical(targets) 

In [30]:
X_train, X_seed, y_train, y_seed = train_test_split(X, y, test_size=0.2, random_state=16)

In [31]:
def chords_n_notes(Snippet):
    Melody = []
    offset = 0 #Incremental
    for i in Snippet:
        #If it is chord
        if ("." in i or i.isdigit()):
            chord_notes = i.split(".") #Seperating the notes in chord
            notes = [] 
            for j in chord_notes:
                inst_note=int(j)
                note_snip = note.Note(inst_note)            
                notes.append(note_snip)
                chord_snip = chord.Chord(notes)
                chord_snip.offset = offset
                Melody.append(chord_snip)
        # pattern is a note
        else: 
            note_snip = note.Note(i)
            note_snip.offset = offset
            Melody.append(note_snip)
        # increase offset each iteration so that notes do not stack
        offset += 1
    Melody_midi = stream.Stream(Melody)   
    return Melody_midi

In [19]:
def generate_melody(model, Note_Count):
    seed = X_seed[np.random.randint(0,len(X_seed)-1)]
    Music = ""
    Notes_Generated=[]
    for i in range(Note_Count):
        seed = seed.reshape(1,window_size,1)
        prediction = model.predict(seed, verbose=0)[0]
        prediction = np.log(prediction) / 1.0 #diversity
        exp_preds = np.exp(prediction)
        prediction = exp_preds / np.sum(exp_preds)
        index = np.argmax(prediction)
        index_N = index/ float(type_size)   
        Notes_Generated.append(index)
        Music = [reverse_mapping[char] for char in Notes_Generated]
        seed = np.insert(seed[0],len(seed[0]),index_N)
        seed = seed[1:]
    #Now, we have music in form or a list of chords and notes and we want to be a midi file.
    Melody = chords_n_notes(Music)
    Melody_midi = stream.Stream(Melody)   
    return Music,Melody_midi

In [20]:
model_path = '/content/goldberg200e80w64b_adamax.h5'

model = load_model(model_path)

In [ ]:
n_notes_to_generate = 100

music_notes, melody = generate_melody(model, n_notes_to_generate)

In [35]:
#To save the generated melody
melody.write('midi','gv1_melody2.mid')

'gv1_melody2.mid'